In [1]:
import pandas as pd
from ibmpairs import paw, authentication
import json
import numpy as np
import logging
import traceback
import os
import glob
import requests
import sys, traceback, asyncio, aiohttp, warnings

In [2]:
import logging

logging.basicConfig(level=logging.DEBUG)
logging.getLogger('ibm_s3transfer').setLevel(logging.INFO)
logging.getLogger('ibm_botocore').setLevel(logging.INFO)
logging.getLogger('ibm_boto3').setLevel(logging.INFO)
logging.getLogger('urllib3').setLevel(logging.INFO)

In [3]:
logging.basicConfig(level=logging.INFO)
pawLogger = logging.getLogger('ibmpairs.paw')
pawLogger.setLevel(logging.INFO)

with open("ibmpairskey.txt", "r") as f:
    pairs_key = f.read()
    
def authenticate(pairs_key):
    
    pairs_credentials = authentication.OAuth2(api_key=pairs_key)
    auth_header = {'Authorization': f'Bearer {pairs_credentials.jwt_token}'}
    PAIRS_SERVER   = 'https://pairs.res.ibm.com'
    PAIRS_CREDENTIALS = authentication.OAuth2(api_key=pairs_key)
    
    return PAIRS_SERVER, PAIRS_CREDENTIALS

## Create layer

In [ ]:
# Get access token

AUTH_PROVIDER_BASE = 'https://auth-b2b-twc.ibm.com'
PAIRS_API_KEY = pairs_key

auth_response = requests.post(    
    AUTH_PROVIDER_BASE + '/connect/token',
    headers={'Content-Type': 'application/x-www-form-urlencoded'},
    data=[
        ('client_id', 'ibm-pairs'),
        ('grant_type', 'apikey'),
        ('apikey', PAIRS_API_KEY)])

auth_obj = auth_response.json()
access_token = auth_obj.get('access_token')
refresh_token = auth_obj.get('refresh_token')

In [ ]:
with open('dem_layer.json') as data_file:    
    data = json.load(data_file)

for layer in data['layers']:
    json_layer = {
        "layerType": "Raster",
        "layers": [layer]
    }
    dataset_id = '588'
    api_response = requests.post(
        'https://pairs.res.ibm.com/v2/datasets/{}/datalayers'.format(dataset_id),
        headers = {
            'Authorization':'Bearer {}'.format(access_token),
            'Content-Type': 'application/json'
        },
        json = json_layer
    )
    print(api_response)

# Uploading to IBM PAIRS

This notebook illustrates how to use the `uploads` module to upload to IBM PAIRS via IBM Cloud Object Store (COS).

In [4]:
#from ibmpairs import uploads
import uploads_key_auth as uploads #local script
import logging
from pathlib import Path

We want full debug logging for the uploads module.

In [5]:
#logging.basicConfig(level=logging.DEBUG)

However, we want these other modules to be quiet.

In [6]:
logging.getLogger('ibm_s3transfer').setLevel(logging.INFO)
logging.getLogger('ibm_botocore').setLevel(logging.INFO)
logging.getLogger('ibm_boto3').setLevel(logging.INFO)
logging.getLogger('urllib3').setLevel(logging.INFO)

## Credentials

Not surprisingly, we need a whole host of credentials. The first 7 of these are COS related, the final is just PAIRS username&password.

In [7]:
apikey = "FzmaPxVOTb6_LOH1AqNggpFwER7n61EISx1_TqrQugKU"
resourceInstanceID = "crn:v1:bluemix:public:cloud-object-storage:global:a/045077ab9f1a4dcfafa2c58389d3d639:11a6c883-43a0-4758-a264-6568f2b5ad9a::"
authEndpoint = "https://iam.eu-de.bluemix.net/oidc/token"
endpoint = "https://s3.eu-de.cloud-object-storage.appdomain.cloud"
bucketName = "drought"
accessKeyID = "b9f5af1d0bae45e4bbd9a58420a6190c"
secretAccessKey = "53faf26559acb6e0752d16d869cabff2016a1991b97d216a"

In [8]:
# Get access token

AUTH_PROVIDER_BASE = 'https://auth-b2b-twc.ibm.com'
PAIRS_API_KEY = pairs_key

auth_response = requests.post(    
    AUTH_PROVIDER_BASE + '/connect/token',
    headers={'Content-Type': 'application/x-www-form-urlencoded'},
    data=[
        ('client_id', 'ibm-pairs'),
        ('grant_type', 'apikey'),
        ('apikey', PAIRS_API_KEY)])

auth_obj = auth_response.json()
access_token = auth_obj.get('access_token')
refresh_token = auth_obj.get('refresh_token')

pairsAuthHeader = {
        'Authorization':'Bearer {}'.format(access_token),
        'Content-Type': 'application/json'
    }

## Uploading canton by canton

In [9]:
# Get name from urls txt
# upload file from demfolder

In [9]:
""" VARIABLES """
URLS_PATH = 'swiss_dem_urls/jura.txt'
DOWNLOADS_PATH = 'dem_reprojected/'
layer_id = "51726"


""" UPLOAD """

data = pd.read_csv(URLS_PATH, header=None)
data.columns = ["link"]

for idx, row in data.iterrows():
    if idx >= 460:
        print('File', idx)
        url = row["link"]
        name = url.split('/')[-1].split('\n')[0] 
        
        ### Reauthenticate ###
        AUTH_PROVIDER_BASE = 'https://auth-b2b-twc.ibm.com'
        PAIRS_API_KEY = pairs_key

        auth_response = requests.post(    
            AUTH_PROVIDER_BASE + '/connect/token',
            headers={'Content-Type': 'application/x-www-form-urlencoded'},
            data=[
                ('client_id', 'ibm-pairs'),
                ('grant_type', 'apikey'),
                ('apikey', PAIRS_API_KEY)])

        auth_obj = auth_response.json()
        access_token = auth_obj.get('access_token')
        refresh_token = auth_obj.get('refresh_token')

        pairsAuthHeader = {
                'Authorization':'Bearer {}'.format(access_token),
                'Content-Type': 'application/json'
            }

        
        ### Create meta.json ###
        meta_json = {
            "datalayer_id": [layer_id],
            "pairsdatatype": "2draster",
            "datainterpolation": "bilinear",
            "timestamp": "20220101T000000Z"
        }


        json_object = json.dumps(meta_json, indent=4)

        with open(DOWNLOADS_PATH + f"{name}.meta.json", "w") as outfile:
            outfile.write(json_object)

        ### Upload to COS and PAIRS ###
        cosBucket = uploads.IBMCosBucket(
            apikey, resourceInstanceID, authEndpoint, endpoint,
            bucketName, accessKeyID, secretAccessKey
        )

        filesToUpload = [DOWNLOADS_PATH + name]

        uploadFromLocal = uploads.PAIRSUpload.fromLocal(
            cosBucket, pairsAuthHeader, False, fileList=filesToUpload
        )

        await uploadFromLocal.run(1)

        # add a check to see if uploaded correctly
        if {v.remoteObject for v in uploadFromLocal.completedUploads if not v.status == 'SUCCEEDED'}:
            # Some file weren't uploaded correctly
            print('File failed', idx)
            break

        ### delete meta.json ###

        fileList = glob.glob(DOWNLOADS_PATH + '*.meta.json', recursive=True)
        for filePath in fileList:
            os.remove(filePath)

        cosBucket.delete(name)
    

File 460


INFO:uploads_key_auth:'PAIRSUpload'. Calling 'run' will upload 1 files. Total raw data volume 935686 bytes. Depending on the PAIRS resolution level of target layer(s), the data volume might be significantly different in IBM PAIRS.
DEBUG:uploads_key_auth:Commencing upload run.
DEBUG:uploads_key_auth:Initialized ClientSession.
DEBUG:uploads_key_auth:Worker picked up dem_reprojected/swissalti3d_2019_2575-1252_2_2056_5728.tif.
DEBUG:uploads_key_auth:Pushing swissalti3d_2019_2575-1252_2_2056_5728.tif to COS.
DEBUG:uploads_key_auth:Pushing swissalti3d_2019_2575-1252_2_2056_5728.tif.meta.json to COS.
DEBUG:uploads_key_auth:Worker moved swissalti3d_2019_2575-1252_2_2056_5728.tif to COS.
DEBUG:uploads_key_auth:Worker will obtain meta data for swissalti3d_2019_2575-1252_2_2056_5728.tif.
DEBUG:uploads_key_auth:Worker has obtained meta data for swissalti3d_2019_2575-1252_2_2056_5728.tif.
DEBUG:uploads_key_auth:Starting PAIRS upload.
DEBUG:uploads_key_auth:Submitting upload request to PAIRS.
DEBUG:

File 461


INFO:uploads_key_auth:'PAIRSUpload'. Calling 'run' will upload 1 files. Total raw data volume 935686 bytes. Depending on the PAIRS resolution level of target layer(s), the data volume might be significantly different in IBM PAIRS.
DEBUG:uploads_key_auth:Commencing upload run.
DEBUG:uploads_key_auth:Initialized ClientSession.
DEBUG:uploads_key_auth:Worker picked up dem_reprojected/swissalti3d_2019_2575-1253_2_2056_5728.tif.
DEBUG:uploads_key_auth:Pushing swissalti3d_2019_2575-1253_2_2056_5728.tif to COS.
DEBUG:uploads_key_auth:Pushing swissalti3d_2019_2575-1253_2_2056_5728.tif.meta.json to COS.
DEBUG:uploads_key_auth:Worker moved swissalti3d_2019_2575-1253_2_2056_5728.tif to COS.
DEBUG:uploads_key_auth:Worker will obtain meta data for swissalti3d_2019_2575-1253_2_2056_5728.tif.
DEBUG:uploads_key_auth:Worker has obtained meta data for swissalti3d_2019_2575-1253_2_2056_5728.tif.
DEBUG:uploads_key_auth:Starting PAIRS upload.
DEBUG:uploads_key_auth:Submitting upload request to PAIRS.
DEBUG:

File 462


...

INFO:uploads_key_auth:'PAIRSUpload'. Calling 'run' will upload 1 files. Total raw data volume 940388 bytes. Depending on the PAIRS resolution level of target layer(s), the data volume might be significantly different in IBM PAIRS.
DEBUG:uploads_key_auth:Commencing upload run.
DEBUG:uploads_key_auth:Initialized ClientSession.
DEBUG:uploads_key_auth:Worker picked up dem_reprojected/swissalti3d_2021_2556-1223_2_2056_5728.tif.
DEBUG:uploads_key_auth:Pushing swissalti3d_2021_2556-1223_2_2056_5728.tif to COS.
DEBUG:uploads_key_auth:Pushing swissalti3d_2021_2556-1223_2_2056_5728.tif.meta.json to COS.
DEBUG:uploads_key_auth:Worker moved swissalti3d_2021_2556-1223_2_2056_5728.tif to COS.
DEBUG:uploads_key_auth:Worker will obtain meta data for swissalti3d_2021_2556-1223_2_2056_5728.tif.
DEBUG:uploads_key_auth:Worker has obtained meta data for swissalti3d_2021_2556-1223_2_2056_5728.tif.
DEBUG:uploads_key_auth:Starting PAIRS upload.
DEBUG:uploads_key_auth:Submitting upload request to PAIRS.
DEBUG:

File 969


INFO:uploads_key_auth:'PAIRSUpload'. Calling 'run' will upload 1 files. Total raw data volume 940388 bytes. Depending on the PAIRS resolution level of target layer(s), the data volume might be significantly different in IBM PAIRS.
DEBUG:uploads_key_auth:Commencing upload run.
DEBUG:uploads_key_auth:Initialized ClientSession.
DEBUG:uploads_key_auth:Worker picked up dem_reprojected/swissalti3d_2021_2556-1224_2_2056_5728.tif.
DEBUG:uploads_key_auth:Pushing swissalti3d_2021_2556-1224_2_2056_5728.tif to COS.
DEBUG:uploads_key_auth:Pushing swissalti3d_2021_2556-1224_2_2056_5728.tif.meta.json to COS.
DEBUG:uploads_key_auth:Worker moved swissalti3d_2021_2556-1224_2_2056_5728.tif to COS.
DEBUG:uploads_key_auth:Worker will obtain meta data for swissalti3d_2021_2556-1224_2_2056_5728.tif.
DEBUG:uploads_key_auth:Worker has obtained meta data for swissalti3d_2021_2556-1224_2_2056_5728.tif.
DEBUG:uploads_key_auth:Starting PAIRS upload.
DEBUG:uploads_key_auth:Submitting upload request to PAIRS.
DEBUG:

File 970


INFO:uploads_key_auth:'PAIRSUpload'. Calling 'run' will upload 1 files. Total raw data volume 940388 bytes. Depending on the PAIRS resolution level of target layer(s), the data volume might be significantly different in IBM PAIRS.
DEBUG:uploads_key_auth:Commencing upload run.
DEBUG:uploads_key_auth:Initialized ClientSession.
DEBUG:uploads_key_auth:Worker picked up dem_reprojected/swissalti3d_2021_2557-1222_2_2056_5728.tif.
DEBUG:uploads_key_auth:Pushing swissalti3d_2021_2557-1222_2_2056_5728.tif to COS.
DEBUG:uploads_key_auth:Pushing swissalti3d_2021_2557-1222_2_2056_5728.tif.meta.json to COS.
DEBUG:uploads_key_auth:Worker moved swissalti3d_2021_2557-1222_2_2056_5728.tif to COS.
DEBUG:uploads_key_auth:Worker will obtain meta data for swissalti3d_2021_2557-1222_2_2056_5728.tif.
DEBUG:uploads_key_auth:Worker has obtained meta data for swissalti3d_2021_2557-1222_2_2056_5728.tif.
DEBUG:uploads_key_auth:Starting PAIRS upload.
DEBUG:uploads_key_auth:Submitting upload request to PAIRS.
DEBUG:

In [10]:
filesToUpload

['dem_reprojected/swissalti3d_2022_2630-1229_2_2056_5728.tif']

## Uploading

In [ ]:
# Your root directory path
rootdir = r"dem_reprojected/"
layer_id = "51707"

#Your batch size
batch_size = 8

def walk_dirs(directory, batch_size):
    walk_dirs_generator = os.walk(directory)
    for dirname, subdirectories, filenames in walk_dirs_generator:
        for i in range(0, len(filenames), batch_size):
            # slice the filenames list 0-7, 8-15 and so on
            yield [os.path.join(dirname, filename) for filename in filenames[i:i+batch_size]]

In [ ]:
# Finally iterate over the walk_dirs function which itself returns a generator
batch_nbr = 0
for file_name_batch in walk_dirs(rootdir, batch_size):
    
    if batch_nbr >= 997: # put the batch nbr you want to start at
        print('Batch nbr', batch_nbr)
        print(file_name_batch)

        ### Reauthenticate ###
        AUTH_PROVIDER_BASE = 'https://auth-b2b-twc.ibm.com'
        PAIRS_API_KEY = pairs_key

        auth_response = requests.post(    
            AUTH_PROVIDER_BASE + '/connect/token',
            headers={'Content-Type': 'application/x-www-form-urlencoded'},
            data=[
                ('client_id', 'ibm-pairs'),
                ('grant_type', 'apikey'),
                ('apikey', PAIRS_API_KEY)])

        auth_obj = auth_response.json()
        access_token = auth_obj.get('access_token')
        refresh_token = auth_obj.get('refresh_token')

        pairsAuthHeader = {
                'Authorization':'Bearer {}'.format(access_token),
                'Content-Type': 'application/json'
            }


        file_name_batch = [f for f in file_name_batch if '.tif' in f]

        for file_name in file_name_batch:
            ### Create meta.json ###
            meta_json = {
                "datalayer_id": [layer_id],
                "pairsdatatype": "2draster",
                "datainterpolation": "bilinear",
                "timestamp": "20220101T000000Z"
            }

            json_object = json.dumps(meta_json, indent=4)

            with open(f"{file_name}.meta.json", "w") as outfile:
                outfile.write(json_object)

        ### Upload to COS and PAIRS ###
        cosBucket = uploads.IBMCosBucket(
            apikey, resourceInstanceID, authEndpoint, endpoint,
            bucketName, accessKeyID, secretAccessKey
        )

        filesToUpload = file_name_batch

        uploadFromLocal = uploads.PAIRSUpload.fromLocal(
            cosBucket, pairsAuthHeader, False, fileList=filesToUpload
        )

        await uploadFromLocal.run(8)

        # add a check to see if uploaded correctly
        if {v.remoteObject for v in uploadFromLocal.completedUploads if not v.status == 'SUCCEEDED'}:
            # Some file weren't uploaded correctly
            print('Batch nbr failed', batch_nbr)
            break

        ### delete meta.json ###

        fileList = glob.glob(rootdir + '*.meta.json', recursive=True)
        for filePath in fileList:
            os.remove(filePath)

        for file_name in file_name_batch:
            cosBucket.delete(file_name.split('/')[1])
        
    batch_nbr += 1
        

     
# TO DO edit code so that can restart at any point: use batch_nbr with if condition
# Add checks to see if upload was successful
# remove file from COS